In [1]:
%%script false --no-raise-error
import zarr
from netCDF4 import Dataset
if __name__ == '__main__':
    #Todo read up on hdf5, zarr, netcdf, xarray, mpi, lustre, ceph
    # and check back with dkrz their structure, code conventions, datasets

    # Todo setup test cases proactively to ensure proper comparability and best practice

    #Todo use netcdf dataset for testing (for now) should already bet hdf5
    dest_hdf = Dataset("data/source.nc", "w", format="NETCDF4")
    print(dest_hdf.data_model)
    dest_hdf.close()

    #Todo import netcdf data through hdf5 to zarr (for now)
    dest_zarr = zarr.open_group('data/example2.zarr', mode='w')
    zarr.copy_all(dest_hdf, dest_zarr)
    dest_zarr.tree()

    #Todo conversion to netcdf

    #for hdf5 nothing needs to be done

    #for zarr (wip). netcdf is developing its own implementation
    # but that isn't available yet so I will work on something myself in the meantime

    #Todo setup Lustre and Ceph (need info on that)

    #Todo setup benchmark (prob compression, filesize, access-time, r/w-time) for sequential access

    #Todo setup benchmark for parallel access

    #Todo setup benchmark for random access

    #Todo setup benchmark for parallel with subfileing and async / I/O


In [2]:
%%script false --no-raise-error
from mpi4py import MPI
import numpy as np
import h5py

rank = MPI.COMM_WORLD.rank

# create HDF5 file
with h5py.File('data/test_ds.hdf5', 'w', driver="mpio", comm=MPI.COMM_WORLD) as hf:
    u = hf.create_dataset("u", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    v = hf.create_dataset("v", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    w = hf.create_dataset("w", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    x = hf.create_dataset("x", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    y = hf.create_dataset("y", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)

hf.close()   

In [ ]:
%%time
import xarray as xr
import numpy as np

u = np.random.rand(1_000_000, 10, 10, 1)
v = np.random.rand(1_000_000, 10, 10, 1)
w = np.random.rand(1_000_000, 10, 10, 1)
x = np.random.rand(1_000_000, 10, 10, 1)
y = np.random.rand(1_000_000, 10, 10, 1)
z = np.random.rand(1_000_000, 10, 10, 1)

ds = xr.Dataset(data_vars=dict(
                            u=(["1","2","3","4"], u),
                            v=(["1","2","3","4"], v),
                            w=(["1","2","3","4"], w),
                            x=(["1","2","3","4"], x),
                            y=(["1","2","3","4"], y),
                            z=(["1","2","3","4"], z)
                            ))

ds.to_netcdf('data/test_dataset.nc', mode="w")

ds.to_netcdf('data/test_dataset.hdf5', mode="w", engine="h5netcdf")

ds.to_zarr("data/test_dataset.zarr", mode="w")



KeyboardInterrupt: 

: 

In [1]:
%%time 
import xarray as xr
ds_zarr = xr.open_zarr('data/test_dataset.zarr', consolidated=True)


CPU times: user 1.59 s, sys: 57.3 ms, total: 1.64 s
Wall time: 336 ms


In [2]:
%%time 
import xarray as xr
max_var = ds_zarr["x"].max().compute()


CPU times: user 468 ms, sys: 551 ms, total: 1.02 s
Wall time: 434 ms


In [35]:
%%time
import xarray as xr
ds_netcdf4 = xr.open_dataset(filename_or_obj="data/test_dataset.nc", engine="h5netcdf")


CPU times: user 9.27 ms, sys: 0 ns, total: 9.27 ms
Wall time: 7.95 ms


In [ ]:
%%time
import xarray as xr
max_var = ds_netcdf4["x"].max().compute()

CPU times: user 15 ms, sys: 2.8 ms, total: 17.8 ms
Wall time: 15.7 ms
